# Fundamentals of Generative Models

## Learning Objectives
- Understand probability distributions and their role in generative modeling
- Learn about maximum likelihood estimation (MLE)
- Explore latent variable models
- Implement basic generative models from scratch
- Understand sampling techniques

## What Makes a Model "Generative"?

A generative model learns the joint probability distribution P(X, Y) or the data distribution P(X), allowing it to:
1. **Generate new samples** that look like the training data
2. **Estimate probabilities** of new data points
3. **Handle missing data** through conditional distributions
4. **Provide uncertainty estimates** for predictions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs, make_moons
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

## 1. Probability Distributions: The Foundation

### Understanding Data Distributions

In [ ]:
# Generate sample data from different distributions
n_samples = 1000

# 1. Normal distribution
normal_data = np.random.normal(0, 1, n_samples)

# 2. Bimodal distribution (mixture of two normals)
bimodal_data = np.concatenate([
    np.random.normal(-2, 0.5, n_samples//2),
    np.random.normal(2, 0.5, n_samples//2)
])

# 3. Exponential distribution
exp_data = np.random.exponential(1, n_samples)

# 4. Uniform distribution
uniform_data = np.random.uniform(-3, 3, n_samples)

# Visualize different distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

distributions = [
    (normal_data, 'Normal Distribution', axes[0, 0]),
    (bimodal_data, 'Bimodal Distribution', axes[0, 1]),
    (exp_data, 'Exponential Distribution', axes[1, 0]),
    (uniform_data, 'Uniform Distribution', axes[1, 1])
]

for data, title, ax in distributions:
    ax.hist(data, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    ax.set_title(title)
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Statistics:")
for data, name in zip([normal_data, bimodal_data, exp_data, uniform_data], 
                     ['Normal', 'Bimodal', 'Exponential', 'Uniform']):
    print(f"{name:12} - Mean: {np.mean(data):.2f}, Std: {np.std(data):.2f}")

## 2. Maximum Likelihood Estimation (MLE)

MLE finds the parameters θ that maximize the likelihood of observing the data:

$$\theta^* = \arg\max_\theta \prod_{i=1}^{n} P(x_i | \theta) = \arg\max_\theta \sum_{i=1}^{n} \log P(x_i | \theta)$$

In [ ]:
def gaussian_likelihood(data, mu, sigma):
    """Calculate likelihood of data under Gaussian distribution"""
    n = len(data)
    log_likelihood = -n/2 * np.log(2 * np.pi * sigma**2) - np.sum((data - mu)**2) / (2 * sigma**2)
    return log_likelihood

def mle_gaussian(data):
    """Maximum Likelihood Estimation for Gaussian distribution"""
    mu_mle = np.mean(data)
    sigma_mle = np.std(data, ddof=0)  # MLE uses N, not N-1
    return mu_mle, sigma_mle

# Generate sample data
true_mu, true_sigma = 2.5, 1.2
sample_data = np.random.normal(true_mu, true_sigma, 500)

# Estimate parameters using MLE
estimated_mu, estimated_sigma = mle_gaussian(sample_data)

print(f"True parameters:      μ = {true_mu:.2f}, σ = {true_sigma:.2f}")
print(f"MLE estimates:        μ = {estimated_mu:.2f}, σ = {estimated_sigma:.2f}")
print(f"Sample mean/std:      μ = {np.mean(sample_data):.2f}, σ = {np.std(sample_data):.2f}")

# Visualize likelihood surface
mu_range = np.linspace(1.5, 3.5, 50)
sigma_range = np.linspace(0.8, 1.8, 50)
MU, SIGMA = np.meshgrid(mu_range, sigma_range)

# Calculate likelihood for each parameter combination
likelihood_surface = np.zeros_like(MU)
for i in range(len(mu_range)):
    for j in range(len(sigma_range)):
        likelihood_surface[j, i] = gaussian_likelihood(sample_data, MU[j, i], SIGMA[j, i])

# Plot likelihood surface
fig = plt.figure(figsize=(15, 5))

# 3D surface plot
ax1 = fig.add_subplot(131, projection='3d')
surf = ax1.plot_surface(MU, SIGMA, likelihood_surface, cmap='viridis', alpha=0.8)
ax1.scatter([estimated_mu], [estimated_sigma], [gaussian_likelihood(sample_data, estimated_mu, estimated_sigma)], 
           color='red', s=100, label='MLE Estimate')
ax1.set_xlabel('μ')
ax1.set_ylabel('σ')
ax1.set_zlabel('Log-Likelihood')
ax1.set_title('Likelihood Surface')

# Contour plot
ax2 = fig.add_subplot(132)
contour = ax2.contour(MU, SIGMA, likelihood_surface, levels=20)
ax2.scatter(estimated_mu, estimated_sigma, color='red', s=100, marker='x', linewidth=3, label='MLE Estimate')
ax2.scatter(true_mu, true_sigma, color='green', s=100, marker='o', label='True Parameters')
ax2.set_xlabel('μ')
ax2.set_ylabel('σ')
ax2.set_title('Likelihood Contours')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Data histogram with fitted distribution
ax3 = fig.add_subplot(133)
ax3.hist(sample_data, bins=30, density=True, alpha=0.7, color='lightblue', edgecolor='black', label='Data')
x_range = np.linspace(sample_data.min(), sample_data.max(), 100)
true_pdf = stats.norm.pdf(x_range, true_mu, true_sigma)
estimated_pdf = stats.norm.pdf(x_range, estimated_mu, estimated_sigma)
ax3.plot(x_range, true_pdf, 'g-', linewidth=2, label=f'True: N({true_mu:.1f}, {true_sigma:.1f})')
ax3.plot(x_range, estimated_pdf, 'r--', linewidth=2, label=f'MLE: N({estimated_mu:.1f}, {estimated_sigma:.1f})')
ax3.set_xlabel('Value')
ax3.set_ylabel('Density')
ax3.set_title('Fitted Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Gaussian Mixture Models (GMM)

GMMs model data as a mixture of multiple Gaussian distributions:

$$P(x) = \sum_{k=1}^{K} \pi_k \mathcal{N}(x | \mu_k, \Sigma_k)$$

In [ ]:
class SimpleGMM:
    def __init__(self, n_components=2, max_iter=100, tol=1e-6):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        
    def fit(self, X):
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.weights = np.ones(self.n_components) / self.n_components
        self.means = X[np.random.choice(n_samples, self.n_components, replace=False)]
        self.covariances = [np.eye(n_features) for _ in range(self.n_components)]
        
        log_likelihood_old = -np.inf
        
        for iteration in range(self.max_iter):
            # E-step: Calculate responsibilities
            responsibilities = self._e_step(X)
            
            # M-step: Update parameters
            self._m_step(X, responsibilities)
            
            # Check convergence
            log_likelihood = self._log_likelihood(X)
            if abs(log_likelihood - log_likelihood_old) < self.tol:
                print(f"Converged after {iteration + 1} iterations")
                break
            log_likelihood_old = log_likelihood
        
        return self
    
    def _e_step(self, X):
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_components))
        
        for k in range(self.n_components):
            responsibilities[:, k] = self.weights[k] * stats.multivariate_normal.pdf(
                X, self.means[k], self.covariances[k]
            )
        
        # Normalize responsibilities
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        return responsibilities
    
    def _m_step(self, X, responsibilities):
        n_samples = X.shape[0]
        
        for k in range(self.n_components):
            # Update weights
            Nk = responsibilities[:, k].sum()
            self.weights[k] = Nk / n_samples
            
            # Update means
            self.means[k] = (responsibilities[:, k][:, np.newaxis] * X).sum(axis=0) / Nk
            
            # Update covariances
            diff = X - self.means[k]
            self.covariances[k] = np.dot(
                (responsibilities[:, k][:, np.newaxis] * diff).T, diff
            ) / Nk
    
    def _log_likelihood(self, X):
        likelihood = np.zeros(X.shape[0])
        for k in range(self.n_components):
            likelihood += self.weights[k] * stats.multivariate_normal.pdf(
                X, self.means[k], self.covariances[k]
            )
        return np.sum(np.log(likelihood))
    
    def sample(self, n_samples):
        """Generate samples from the fitted GMM"""
        # Choose components based on weights
        components = np.random.choice(self.n_components, n_samples, p=self.weights)
        
        samples = []
        for i in range(n_samples):
            k = components[i]
            sample = np.random.multivariate_normal(self.means[k], self.covariances[k])
            samples.append(sample)
        
        return np.array(samples)

# Generate sample data (mixture of 3 Gaussians)
np.random.seed(42)
X, y_true = make_blobs(n_samples=300, centers=3, n_features=2, 
                       random_state=42, cluster_std=1.5)

# Fit our GMM
gmm_custom = SimpleGMM(n_components=3)
gmm_custom.fit(X)

# Fit sklearn GMM for comparison
gmm_sklearn = GaussianMixture(n_components=3, random_state=42)
gmm_sklearn.fit(X)

# Generate samples from both models
samples_custom = gmm_custom.sample(300)
samples_sklearn, _ = gmm_sklearn.sample(300)

# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original data
axes[0, 0].scatter(X[:, 0], X[:, 1], c=y_true, alpha=0.7)
axes[0, 0].set_title('Original Data')
axes[0, 0].grid(True, alpha=0.3)

# Custom GMM samples
axes[0, 1].scatter(samples_custom[:, 0], samples_custom[:, 1], alpha=0.7, color='red')
axes[0, 1].set_title('Custom GMM Samples')
axes[0, 1].grid(True, alpha=0.3)

# Sklearn GMM samples
axes[1, 0].scatter(samples_sklearn[:, 0], samples_sklearn[:, 1], alpha=0.7, color='green')
axes[1, 0].set_title('Sklearn GMM Samples')
axes[1, 0].grid(True, alpha=0.3)

# Comparison
axes[1, 1].scatter(X[:, 0], X[:, 1], alpha=0.5, label='Original', color='blue')
axes[1, 1].scatter(samples_custom[:, 0], samples_custom[:, 1], alpha=0.5, label='Custom GMM', color='red')
axes[1, 1].scatter(samples_sklearn[:, 0], samples_sklearn[:, 1], alpha=0.5, label='Sklearn GMM', color='green')
axes[1, 1].set_title('Comparison')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nModel Parameters:")
print(f"Custom GMM weights: {gmm_custom.weights}")
print(f"Sklearn GMM weights: {gmm_sklearn.weights_}")

## 4. Latent Variable Models

Latent variable models assume that observed data is generated from hidden (latent) variables:

$$P(x) = \int P(x|z) P(z) dz$$

In [ ]:
class SimpleLatentVariableModel:
    def __init__(self, latent_dim=2, observed_dim=4):
        self.latent_dim = latent_dim
        self.observed_dim = observed_dim
        
        # Initialize parameters
        self.W = np.random.randn(observed_dim, latent_dim) * 0.5  # Loading matrix
        self.mu = np.zeros(observed_dim)  # Mean
        self.sigma2 = 1.0  # Noise variance
    
    def generate_data(self, n_samples):
        """Generate data from the latent variable model"""
        # Sample latent variables
        Z = np.random.randn(n_samples, self.latent_dim)
        
        # Generate observations
        X = Z @ self.W.T + self.mu + np.random.randn(n_samples, self.observed_dim) * np.sqrt(self.sigma2)
        
        return X, Z
    
    def fit(self, X, max_iter=100):
        """Fit the model using EM algorithm (simplified)"""
        n_samples = X.shape[0]
        
        for iteration in range(max_iter):
            # E-step: Compute posterior of latent variables
            # For simplicity, we'll use a closed-form solution for linear Gaussian models
            
            # M-step: Update parameters
            # This is a simplified version - full EM would be more complex
            
            # Update mean
            self.mu = np.mean(X, axis=0)
            
            # Center the data
            X_centered = X - self.mu
            
            # Update W using PCA-like approach
            U, s, Vt = np.linalg.svd(X_centered, full_matrices=False)
            self.W = Vt[:self.latent_dim].T * np.sqrt(s[:self.latent_dim] / n_samples)
            
            # Update noise variance
            reconstruction = (X_centered @ self.W) @ self.W.T
            self.sigma2 = np.mean((X_centered - reconstruction) ** 2)
        
        return self
    
    def transform(self, X):
        """Transform data to latent space"""
        X_centered = X - self.mu
        # Simplified posterior mean
        M = self.W.T @ self.W + self.sigma2 * np.eye(self.latent_dim)
        Z_mean = X_centered @ self.W @ np.linalg.inv(M)
        return Z_mean
    
    def reconstruct(self, Z):
        """Reconstruct data from latent variables"""
        return Z @ self.W.T + self.mu

# Generate synthetic data
true_model = SimpleLatentVariableModel(latent_dim=2, observed_dim=4)
X_true, Z_true = true_model.generate_data(500)

# Fit model to the data
learned_model = SimpleLatentVariableModel(latent_dim=2, observed_dim=4)
learned_model.fit(X_true)

# Transform data to latent space
Z_inferred = learned_model.transform(X_true)

# Reconstruct data
X_reconstructed = learned_model.reconstruct(Z_inferred)

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# True latent variables
axes[0, 0].scatter(Z_true[:, 0], Z_true[:, 1], alpha=0.7, color='blue')
axes[0, 0].set_title('True Latent Variables')
axes[0, 0].set_xlabel('Z1')
axes[0, 0].set_ylabel('Z2')
axes[0, 0].grid(True, alpha=0.3)

# Inferred latent variables
axes[0, 1].scatter(Z_inferred[:, 0], Z_inferred[:, 1], alpha=0.7, color='red')
axes[0, 1].set_title('Inferred Latent Variables')
axes[0, 1].set_xlabel('Z1')
axes[0, 1].set_ylabel('Z2')
axes[0, 1].grid(True, alpha=0.3)

# Comparison of latent variables
axes[0, 2].scatter(Z_true[:, 0], Z_inferred[:, 0], alpha=0.7, label='Dimension 1')
axes[0, 2].scatter(Z_true[:, 1], Z_inferred[:, 1], alpha=0.7, label='Dimension 2')
axes[0, 2].plot([-3, 3], [-3, 3], 'k--', alpha=0.5)
axes[0, 2].set_title('True vs Inferred Latent Variables')
axes[0, 2].set_xlabel('True Z')
axes[0, 2].set_ylabel('Inferred Z')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Original data (first two dimensions)
axes[1, 0].scatter(X_true[:, 0], X_true[:, 1], alpha=0.7, color='blue')
axes[1, 0].set_title('Original Data (Dim 1 vs 2)')
axes[1, 0].set_xlabel('X1')
axes[1, 0].set_ylabel('X2')
axes[1, 0].grid(True, alpha=0.3)

# Reconstructed data (first two dimensions)
axes[1, 1].scatter(X_reconstructed[:, 0], X_reconstructed[:, 1], alpha=0.7, color='red')
axes[1, 1].set_title('Reconstructed Data (Dim 1 vs 2)')
axes[1, 1].set_xlabel('X1')
axes[1, 1].set_ylabel('X2')
axes[1, 1].grid(True, alpha=0.3)

# Reconstruction error
reconstruction_error = np.mean((X_true - X_reconstructed) ** 2, axis=1)
axes[1, 2].hist(reconstruction_error, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1, 2].set_title('Reconstruction Error Distribution')
axes[1, 2].set_xlabel('Mean Squared Error')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean reconstruction error: {np.mean(reconstruction_error):.4f}")
print(f"Learned noise variance: {learned_model.sigma2:.4f}")
print(f"True noise variance: {true_model.sigma2:.4f}")

## 5. Sampling Techniques

### Different methods to generate samples from distributions

In [ ]:
# 1. Inverse Transform Sampling
def inverse_transform_exponential(n_samples, lam=1.0):
    """Generate exponential samples using inverse transform"""
    u = np.random.uniform(0, 1, n_samples)
    return -np.log(1 - u) / lam

# 2. Rejection Sampling
def rejection_sampling_beta(n_samples, alpha=2, beta=3):
    """Generate Beta distribution samples using rejection sampling"""
    samples = []
    
    # Find maximum of the target distribution for normalization
    x_max = (alpha - 1) / (alpha + beta - 2) if alpha > 1 and beta > 1 else 0.5
    f_max = stats.beta.pdf(x_max, alpha, beta)
    
    while len(samples) < n_samples:
        # Propose from uniform distribution
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, f_max)
        
        # Accept or reject
        if y <= stats.beta.pdf(x, alpha, beta):
            samples.append(x)
    
    return np.array(samples)

# 3. Metropolis-Hastings Sampling
def metropolis_hastings_normal(n_samples, target_mean=0, target_std=1, proposal_std=0.5):
    """Sample from normal distribution using Metropolis-Hastings"""
    samples = []
    current = 0.0  # Starting point
    
    for _ in range(n_samples):
        # Propose new state
        proposal = current + np.random.normal(0, proposal_std)
        
        # Calculate acceptance probability
        current_prob = stats.norm.pdf(current, target_mean, target_std)
        proposal_prob = stats.norm.pdf(proposal, target_mean, target_std)
        
        alpha = min(1, proposal_prob / current_prob)
        
        # Accept or reject
        if np.random.uniform() < alpha:
            current = proposal
        
        samples.append(current)
    
    return np.array(samples)

# Generate samples using different methods
n_samples = 1000

# Exponential distribution
exp_inverse = inverse_transform_exponential(n_samples, lam=1.5)
exp_direct = np.random.exponential(1/1.5, n_samples)

# Beta distribution
beta_rejection = rejection_sampling_beta(n_samples, alpha=2, beta=3)
beta_direct = np.random.beta(2, 3, n_samples)

# Normal distribution
normal_mcmc = metropolis_hastings_normal(n_samples, target_mean=2, target_std=1.5)
normal_direct = np.random.normal(2, 1.5, n_samples)

# Visualize sampling results
fig, axes = plt.subplots(3, 2, figsize=(12, 15))

# Exponential distribution
axes[0, 0].hist(exp_inverse, bins=30, alpha=0.7, density=True, label='Inverse Transform', color='blue')
axes[0, 0].hist(exp_direct, bins=30, alpha=0.7, density=True, label='Direct Sampling', color='red')
x_exp = np.linspace(0, 5, 100)
axes[0, 0].plot(x_exp, stats.expon.pdf(x_exp, scale=1/1.5), 'k-', linewidth=2, label='True PDF')
axes[0, 0].set_title('Exponential Distribution Sampling')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Beta distribution
axes[1, 0].hist(beta_rejection, bins=30, alpha=0.7, density=True, label='Rejection Sampling', color='blue')
axes[1, 0].hist(beta_direct, bins=30, alpha=0.7, density=True, label='Direct Sampling', color='red')
x_beta = np.linspace(0, 1, 100)
axes[1, 0].plot(x_beta, stats.beta.pdf(x_beta, 2, 3), 'k-', linewidth=2, label='True PDF')
axes[1, 0].set_title('Beta Distribution Sampling')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Normal distribution
axes[2, 0].hist(normal_mcmc, bins=30, alpha=0.7, density=True, label='MCMC Sampling', color='blue')
axes[2, 0].hist(normal_direct, bins=30, alpha=0.7, density=True, label='Direct Sampling', color='red')
x_norm = np.linspace(-2, 6, 100)
axes[2, 0].plot(x_norm, stats.norm.pdf(x_norm, 2, 1.5), 'k-', linewidth=2, label='True PDF')
axes[2, 0].set_title('Normal Distribution Sampling')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# MCMC trace plot
axes[0, 1].plot(normal_mcmc[:200])
axes[0, 1].set_title('MCMC Trace Plot (First 200 samples)')
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Sample Value')
axes[0, 1].grid(True, alpha=0.3)

# Autocorrelation
def autocorrelation(x, max_lag=50):
    n = len(x)
    x = x - np.mean(x)
    autocorr = np.correlate(x, x, mode='full')
    autocorr = autocorr[n-1:n-1+max_lag]
    return autocorr / autocorr[0]

lags = range(50)
autocorr = autocorrelation(normal_mcmc)
axes[1, 1].plot(lags, autocorr)
axes[1, 1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
axes[1, 1].set_title('MCMC Autocorrelation')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('Autocorrelation')
axes[1, 1].grid(True, alpha=0.3)

# Q-Q plots for comparison
from scipy import stats as scipy_stats
scipy_stats.probplot(normal_mcmc, dist="norm", sparams=(2, 1.5), plot=axes[2, 1])
axes[2, 1].set_title('Q-Q Plot: MCMC vs True Normal')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print("Sampling Method Comparison:")
print("=" * 40)
print(f"Exponential (λ=1.5):")
print(f"  True mean: {1/1.5:.3f}, Inverse transform: {np.mean(exp_inverse):.3f}, Direct: {np.mean(exp_direct):.3f}")
print(f"\nBeta (α=2, β=3):")
print(f"  True mean: {2/(2+3):.3f}, Rejection: {np.mean(beta_rejection):.3f}, Direct: {np.mean(beta_direct):.3f}")
print(f"\nNormal (μ=2, σ=1.5):")
print(f"  True mean: 2.000, MCMC: {np.mean(normal_mcmc):.3f}, Direct: {np.mean(normal_direct):.3f}")

## 6. Evaluation Metrics for Generative Models

In [ ]:
def calculate_kl_divergence(p, q, epsilon=1e-10):
    """Calculate KL divergence between two discrete distributions"""
    # Add small epsilon to avoid log(0)
    p = p + epsilon
    q = q + epsilon
    
    # Normalize to ensure they sum to 1
    p = p / np.sum(p)
    q = q / np.sum(q)
    
    return np.sum(p * np.log(p / q))

def calculate_js_divergence(p, q):
    """Calculate Jensen-Shannon divergence"""
    m = 0.5 * (p + q)
    return 0.5 * calculate_kl_divergence(p, m) + 0.5 * calculate_kl_divergence(q, m)

def wasserstein_distance_1d(x, y):
    """Calculate 1D Wasserstein distance (Earth Mover's Distance)"""
    x_sorted = np.sort(x)
    y_sorted = np.sort(y)
    
    # Make same length by interpolation
    n = min(len(x_sorted), len(y_sorted))
    x_interp = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, len(x_sorted)), x_sorted)
    y_interp = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, len(y_sorted)), y_sorted)
    
    return np.mean(np.abs(x_interp - y_interp))

# Generate true and synthetic data
np.random.seed(42)
true_data = np.random.normal(0, 1, 1000)

# Different quality synthetic datasets
good_synthetic = np.random.normal(0.1, 1.05, 1000)  # Close to true
bad_synthetic = np.random.normal(0.5, 1.5, 1000)    # Further from true
very_bad_synthetic = np.random.exponential(1, 1000)  # Different distribution

# Calculate histograms for discrete distributions
bins = np.linspace(-4, 4, 50)
true_hist, _ = np.histogram(true_data, bins=bins, density=True)
good_hist, _ = np.histogram(good_synthetic, bins=bins, density=True)
bad_hist, _ = np.histogram(bad_synthetic, bins=bins, density=True)
very_bad_hist, _ = np.histogram(very_bad_synthetic, bins=bins, density=True)

# Normalize histograms
true_hist = true_hist / np.sum(true_hist)
good_hist = good_hist / np.sum(good_hist)
bad_hist = bad_hist / np.sum(bad_hist)
very_bad_hist = very_bad_hist / np.sum(very_bad_hist)

# Calculate metrics
datasets = [
    ("Good Synthetic", good_synthetic, good_hist),
    ("Bad Synthetic", bad_synthetic, bad_hist),
    ("Very Bad Synthetic", very_bad_synthetic, very_bad_hist)
]

print("Generative Model Evaluation Metrics:")
print("=" * 50)
print(f"{'Dataset':<20} {'KL Div':<10} {'JS Div':<10} {'Wasserstein':<12}")
print("-" * 50)

for name, data, hist in datasets:
    kl_div = calculate_kl_divergence(true_hist, hist)
    js_div = calculate_js_divergence(true_hist, hist)
    wasserstein = wasserstein_distance_1d(true_data, data)
    
    print(f"{name:<20} {kl_div:<10.4f} {js_div:<10.4f} {wasserstein:<12.4f}")

# Visualize distributions and metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot distributions
x_range = np.linspace(-4, 6, 100)
axes[0, 0].hist(true_data, bins=30, alpha=0.7, density=True, label='True Data', color='blue')
axes[0, 0].hist(good_synthetic, bins=30, alpha=0.7, density=True, label='Good Synthetic', color='green')
axes[0, 0].set_title('True vs Good Synthetic Data')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(true_data, bins=30, alpha=0.7, density=True, label='True Data', color='blue')
axes[0, 1].hist(bad_synthetic, bins=30, alpha=0.7, density=True, label='Bad Synthetic', color='orange')
axes[0, 1].set_title('True vs Bad Synthetic Data')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].hist(true_data, bins=30, alpha=0.7, density=True, label='True Data', color='blue')
axes[1, 0].hist(very_bad_synthetic, bins=30, alpha=0.7, density=True, label='Very Bad Synthetic', color='red')
axes[1, 0].set_title('True vs Very Bad Synthetic Data')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Metric comparison
metrics_data = {
    'KL Divergence': [calculate_kl_divergence(true_hist, hist) for _, _, hist in datasets],
    'JS Divergence': [calculate_js_divergence(true_hist, hist) for _, _, hist in datasets],
    'Wasserstein': [wasserstein_distance_1d(true_data, data) for _, data, _ in datasets]
}

x_pos = np.arange(len(datasets))
width = 0.25

for i, (metric, values) in enumerate(metrics_data.items()):
    axes[1, 1].bar(x_pos + i * width, values, width, label=metric, alpha=0.8)

axes[1, 1].set_xlabel('Synthetic Datasets')
axes[1, 1].set_ylabel('Metric Value')
axes[1, 1].set_title('Evaluation Metrics Comparison')
axes[1, 1].set_xticks(x_pos + width)
axes[1, 1].set_xticklabels([name for name, _, _ in datasets], rotation=45)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Key Takeaways

### Mathematical Foundations:
1. **Probability Distributions**: Core of generative modeling
2. **Maximum Likelihood Estimation**: Parameter learning principle
3. **Latent Variable Models**: Hidden structure in data
4. **Sampling Methods**: Different ways to generate from distributions

### Practical Insights:
1. **Model Selection**: Choose appropriate model for data structure
2. **Evaluation**: Multiple metrics provide different perspectives
3. **Trade-offs**: Quality vs. diversity vs. computational cost
4. **Convergence**: Iterative algorithms need careful monitoring

### Implementation Skills:
1. **From Scratch**: Understanding underlying mathematics
2. **Library Usage**: Leveraging existing implementations
3. **Evaluation**: Proper assessment of generative quality
4. **Visualization**: Understanding model behavior

## 📝 Exercises

### Beginner:
1. Implement a simple 1D Gaussian mixture model
2. Compare different sampling methods for the same distribution
3. Visualize the effect of different GMM parameters

### Intermediate:
1. Implement the full EM algorithm for GMM
2. Create a factor analysis model (linear latent variable model)
3. Implement different MCMC samplers (Gibbs, Hamiltonian Monte Carlo)

### Advanced:
1. Build a hierarchical Bayesian model
2. Implement variational inference for a latent variable model
3. Compare different divergence measures on real datasets

## 🔗 Next Steps

Now that you understand the fundamentals, you're ready to explore:
1. **Variational Autoencoders (VAEs)**: Neural latent variable models
2. **Generative Adversarial Networks (GANs)**: Adversarial training
3. **Autoregressive Models**: Sequential generation
4. **Diffusion Models**: Denoising-based generation

**Next Notebook**: [Types of Generative Models](./02_types_of_models.ipynb) →